In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [3]:
def build_lstm_model(look_back=1, units=50, dropout_rate=0.2):
    """
    Построение LSTM модели
    look_back - кол-во дней, учитывающих предсказание
    """
    model = Sequential()
    

    model.add(Input(shape=(look_back, 1)))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    
   
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    
   
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    
    
    model.add(Dense(units=1))
    
    # оптимизация adam
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32):
    """
    Обучение модели
    """
    # Колбэки
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    model_checkpoint = ModelCheckpoint(
        'best_lstm_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    return history

SyntaxError: non-default argument follows default argument (1547115745.py, line 1)

In [6]:
def plot_training_history(history):
    """
    Визуализация истории обучения
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    #график потерь
    axes[0].plot(history.history['loss'], label='Train Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title('Model Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    #график MAE
    axes[1].plot(history.history['mae'], label='Train MAE')
    axes[1].plot(history.history['val_mae'], label='Validation MAE')
    axes[1].set_title('Model MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

In [7]:
def make_predictions(model, X_test, scaler):
    """
    Создание предсказаний и обратное преобразование масштаба
    """
    y_pred_scaled = model.predict(X_test)
    
    # Создание массива для обратного преобразования
    # Нужно добавить фиктивные колонки для scaler
    y_pred_for_inverse = np.zeros((len(y_pred_scaled), 1))
    y_test_for_inverse = np.zeros((len(y_test), 1))
    
    y_pred_for_inverse[:, 0] = y_pred_scaled[:, 0]
    y_test_for_inverse[:, 0] = y_test
    
    # Обратное преобразование масштаба
    y_pred = scaler.inverse_transform(y_pred_for_inverse)[:, 0]
    y_test_original = scaler.inverse_transform(y_test_for_inverse)[:, 0]
    
    return y_pred, y_test_original

In [ ]:
def evaluate_predictions(y_true, y_pred):
    """
    Оценка качества предсказаний
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    print("Метрики качества:")
    print(f"MAE (Mean Absolute Error): {mae:.4f}")
    print(f"MSE (Mean Squared Error): {mse:.4f}")
    print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
    print(f"R² Score: {r2:.4f}")
    
    return {
        'mae': mae, 'mse': mse, 'rmse': rmse,
        'mape': mape, 'r2': r2
    }

In [9]:
def predict_next_price(model, scaler, recent_prices, look_back=60):
    """
    Предсказание цены на следующий день
    """
    if len(recent_prices) < look_back:
        raise ValueError(f"Нужно минимум {look_back} предыдущих значений")
    
    # Берем последние look_back цен
    recent_data = np.array(recent_prices[-look_back:]).reshape(-1, 1)
    
    # Масштабирование
    recent_scaled = scaler.transform(recent_data)
    
    # Преобразование в нужный формат
    X_input = recent_scaled.reshape(1, look_back, 1)
    
    # Предсказание
    pred_scaled = model.predict(X_input, verbose=0)[0, 0]
    
    # Обратное преобразование
    pred_price = scaler.inverse_transform([[pred_scaled]])[0, 0]
    
    return pred_price

In [10]:
def lstm_price_prediction_pipeline(df, target_col='Close', look_back=60, test_size=0.2):
    """
    Полный пайплайн LSTM для предсказания цен
    """
    print("Шаг 1: Подготовка данных...")
    X_train, X_test, y_train, y_test, scaler, train_size = prepare_data(
        df, target_col, look_back, test_size
    )
    
    print("Шаг 2: Построение модели...")
    model = build_lstm_model(look_back)
    
    print("Шаг 3: Обучение модели...")
    history = train_model(model, X_train, y_train, X_test, y_test)
    
    print("Шаг 4: Прогнозирование...")
    y_pred, y_test_original = make_predictions(model, X_test, scaler)
    
    print("Шаг 5: Оценка...")
    metrics = evaluate_predictions(y_test_original, y_pred)
    
    print("Шаг 6: Визуализация...")
    plot_training_history(history)
    plot_predictions(y_test_original, y_pred, train_size, look_back, df)
    
    return model, scaler, metrics, history